In [ ]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import time
from tqdm import tqdm
import numpy as np
import random


#設置option
option = webdriver.ChromeOptions()
#不加載圖片
option.add_argument('blink-settings=imagesEnabled=false')

driver = webdriver.Chrome(options=option)

driver.get("https://www.instagram.com/")# 定位搜尋框，前往該頁面
username = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "username"))
    )
password = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "password"))
    )

#等到username和password標籤出來後再繼續執行下個動作
login = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]') #定義登入按鈕的標籤
username.clear()
password.clear()
#清除在該標籤的預設文字
username.send_keys('請出入你的ig帳號')
password.send_keys('請輸入你的ig密碼')
#send_keys為輸入()內的文字進入該標籤
login.click() #點擊登入
time.sleep(4)
#輸入網址導到你要的頁面
driver.get("https://www.instagram.com/explore/tags/%E5%8F%B0%E6%9D%B1%E5%A4%A7%E5%AD%B8/")


#=========================================================================================================

#創建array
first_content = np.array([], dtype=str)
first_date = np.array([], dtype=str)

#點選第一個貼文
click_post = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "_9AhH0"))
    )
click_post.click()
time.sleep(3)


#抓貼文內容
first_content_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "C4VMK"))
    )
#抓貼文時間
post_time = driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div/div[2]/div[2]/a/time').get_attribute('title')
#存入array
first_content = np.append(first_content, first_content_element.text)
first_date = np.append(first_date, post_time)
                          
#===========================================================================================================
#點選右箭頭到下一篇貼文
time.sleep(4)
click_arrow1 = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH,"/html/body/div[6]/div[1]/div/div/div/button/div")))
click_arrow1.click()
time.sleep(3)
#將貼文內容與日期儲存
first_content_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "C4VMK"))
    )
post_time = driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div/div[2]/div[2]/a/time').get_attribute('title')

#將上面兩篇的貼文內容與日期合併
first_content = np.append(first_content, first_content_element.text)
first_date = np.append(first_date, post_time)
#先將first_date儲存
out = pd.DataFrame({'date': first_date, 'contents': first_content})
out.to_excel('專屬黃昇宏的台東大學ig貼文內容first.xlsx', encoding="utf_8_sig")
time.sleep(4)
delay_choices = [15, 13, 17, 20, 10]
#===============================================================================================================
def crawler(x):
    for d in range(1):
        #設定存放內容與日期的變數
        contents = np.array([], dtype=str)
        dates = np.array([], dtype=str)
        for i in tqdm(range(200)):
            
                #try and except
                try:
                    #設定隨機延遲
                    
                    delay = random.choice(delay_choices)
                    #點選右箭頭到下一篇貼文(這裡的右箭頭xpath不一樣所以要重新定義)
                    click_arrow2 = WebDriverWait(driver, 120).until(
                        EC.presence_of_element_located((By.XPATH, "/html/body/div[6]/div[1]/div/div/div[2]/button/div/span"))
                    )
                    click_arrow2.click()
                    # 抓取貼文的content class
                    post_content_element = WebDriverWait(driver, 120).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "C4VMK"))
                    )
                    #抓取日期
                    post_time2 = driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div/div[2]/div[2]/a/time').get_attribute('title')

                    #將資料與日期加入變數
                    contents = np.append(contents, post_content_element.text)
                    dates = np.append(dates, post_time2)
                    time.sleep(delay)
                    
                except:
                    print('第' + str(i) + '個網址錯誤')
                    continue



        #將資料存在csv
        output = pd.DataFrame({'date': dates, 'contents': contents})
        output.to_excel('專屬黃昇宏的台東大學ig貼文內容' + str(x) + '.xlsx', encoding="utf_8_sig")
        #跳出迴圈
        print('download complete')
        
for i in range(10):
    crawler(i)
    time.sleep(300)
            
            

In [ ]:
pd.concat([df1,df2,df3],axis=0, ignore_index=True)